# Models and Evaluation

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

data = pd.read_csv('data_processed/data.csv')
display(data.head())

### Remove unnecessary features (Ids)

In [ ]:
no_ids = [c for c in data.columns if c[-3:] != '_id' and c != 'code']
data = data[no_ids]
# data.drop(['account_frequency', 'gender', 'card_type'], axis=1, inplace=True)

display(data.head())
display(data.info())

In [ ]:
def get_features(df):
    return df.drop('status', axis=1)
def get_target(df):
    return df.drop(df.columns.difference(['status']), axis=1)

results = {}

### Encode

In [ ]:
def encode_data(df, columns):
    from sklearn.preprocessing import OrdinalEncoder
    le = OrdinalEncoder()
    cols = [col for col in columns if col in df.keys()]
    df[cols] = le.set_params(encoded_missing_value=-1).fit_transform(df[cols])
    return df

In [ ]:
categorical_columns = list(data.select_dtypes("object").columns)
data = encode_data(data, categorical_columns)

### Split data

In [ ]:
def split_data(features, target, test_size=0.25, random_state=1):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [ ]:
print('Data shape:',data.shape)
print('Status  1:',data[data['status']==1].shape)
print('Status -1:',data[data['status']==-1].shape)

_X_train, _X_test, _y_train, _y_test = split_data(get_features(data), get_target(data))

print('\nTrain shape:',_y_train.shape)
print('Status ratio:',_y_train[_y_train['status']==1].shape[0],'|',_y_train[_y_train['status']==-1].shape[0])

print('\nTest shape:',_y_test.shape)
print('Status ration:',_y_test[_y_test['status']==1].shape[0],'|',_y_test[_y_test['status']==-1].shape[0])

### Resampling

In [ ]:
def oversample(X_train, y_train):
    from imblearn.over_sampling import SMOTE
    sm = SMOTE(random_state=1, sampling_strategy=1.0)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    return X_train, y_train

In [ ]:
_smote_X_train, _smote_y_train = oversample(_X_train, _y_train)

print('\nTrain shape:',_smote_y_train.shape)
print('Status ratio:',_smote_y_train[_smote_y_train['status']==1].shape[0],'|',_smote_y_train[_smote_y_train['status']==-1].shape[0])

### Scaling

In [ ]:
def normalize_data(X_train, X_test, scaler):
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    return X_train, X_test

def standardize_data(X_train, X_test):
    from sklearn.preprocessing import StandardScaler
    return normalize_data(X_train, X_test, StandardScaler())
def min_max_scaling(X_train, X_test):
    from sklearn.preprocessing import MinMaxScaler
    return normalize_data(X_train, X_test, MinMaxScaler())

### Algorithms

In [ ]:
def add_model(name, model, use_smote=False):
    X_train = _X_train.copy()
    X_test = _X_test.copy()
    smote_X_train = _smote_X_train.copy()
    if (name == 'dtc'):
        X_train.drop(['age_on_loan_request_disc'], inplace=True, axis=1)
        X_test.drop(['age_on_loan_request_disc'], inplace=True, axis=1)
        smote_X_train.drop(['age_on_loan_request_disc'], inplace=True, axis=1)
    else:
        X_train.drop(['age_on_loan_request'], inplace=True, axis=1)
        X_test.drop(['age_on_loan_request'], inplace=True, axis=1)
        smote_X_train.drop(['age_on_loan_request'], inplace=True, axis=1)
        
    # Scaling Normalizations
    # X_train, X_test = standardize_data(X_train, X_test)
    # X_train, X_test = min_max_scaling(X_train, X_test)

    if use_smote: model.fit(smote_X_train, np.ravel(_smote_y_train.values))
    else: model.fit(X_train, np.ravel(_y_train.values))
    
    results[name] = {'model': model, 
                    'X_train': X_train, 
                    'X_test': X_test, 
                    'y_train': np.ravel(_y_train.values), 
                    'y_test': np.ravel(_y_test.values)}

In [ ]:
SMOTE = True

In [ ]:
from sklearn.tree import DecisionTreeClassifier
add_model('dtc',
    DecisionTreeClassifier(),
    use_smote=SMOTE
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
add_model('knn',
    KNeighborsClassifier(),
    use_smote=SMOTE
)

In [ ]:
from sklearn.neural_network import MLPClassifier
add_model('mlp',
    MLPClassifier(),
    use_smote=SMOTE
)

In [ ]:
from sklearn.naive_bayes import GaussianNB
add_model('gnb',
    GaussianNB(),
    use_smote=SMOTE
)

In [ ]:
from sklearn.svm import SVC
add_model('svc',
    SVC(probability=True),
    use_smote=SMOTE
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
add_model('rf',
    RandomForestClassifier(),
    use_smote=SMOTE
)

### Randomized Search

In [ ]:
SEARCH = False

In [ ]:
def add_model_rs(name, model, space={}):
    import json
    from datetime import datetime
    from sklearn.model_selection import RandomizedSearchCV
    X = get_features(data)
    y = np.ravel(get_target(data).values)
    if (name == 'dtc'):
        X.drop(['age_on_loan_request_disc'], inplace=True, axis=1)
    else:
        X.drop(['age_on_loan_request'], inplace=True, axis=1)
        
    # Scaling Normalizations
    # X_train, X_test = standardize_data(X_train, X_test)
    # X_train, X_test = min_max_scaling(X_train, X_test)

    search = RandomizedSearchCV(model, space, n_iter=500, scoring='roc_auc', n_jobs=-1, random_state=0, cv=5)
    search.fit(X, y)
    results[name] = {# 'search': search.best_estimator_,
                    'best_params': search.best_params_,
                    'best_score': search.best_score_}
                    #'X': X,
                    #'y': y}
    with open('random_search_results/'+name+'_'+str(datetime.timestamp(datetime.now()))+'.json', 'w') as f:
        json.dump(results[name], f)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
if SEARCH:
    add_model_rs('dtc',
        DecisionTreeClassifier(),
        space={
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            'min_samples_split': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'max_features': [None, 'auto', 'sqrt', 'log2'],
        }
    )

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
if SEARCH:
    add_model_rs('knn',
        KNeighborsClassifier(),
        space={
            'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
        }
    )

In [ ]:
from sklearn.neural_network import MLPClassifier
if SEARCH:
    add_model_rs('mlp',
        MLPClassifier(),
        space={
            
        }
    )

In [ ]:
from sklearn.naive_bayes import GaussianNB
if SEARCH:
    add_model_rs('gnb',
        GaussianNB(),
        space={
            
        }
    )

In [ ]:
from sklearn.svm import SVC
if SEARCH:
    add_model_rs('svc',
        SVC(probability=True),
        space={
            'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'gamma': ['scale', 'auto'],
            'coef0': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
            'shrinking': [True, False],
            'probability': [True],
            'tol': [0.001, 0.0001, 0.00001],
            'cache_size': [200],
            'class_weight': [None, 'balanced'],
            'verbose': [False],
            'max_iter': [-1],
            'decision_function_shape': ['ovo', 'ovr'],
            'break_ties': [False],
            'random_state': [None],
        }
    )

In [ ]:
from sklearn.ensemble import RandomForestClassifier
if SEARCH:
    add_model_rs('rf',
        RandomForestClassifier(),
        space={
            'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            'min_samples_split': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
            'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'max_features': [None, 'auto', 'sqrt', 'log2'],
            'bootstrap': [True, False],
            'oob_score': [False],
            'n_jobs': [-1],
            'random_state': [None],
            'verbose': [0],
            'warm_start': [False],
            'class_weight': [None, 'balanced'],
            'ccp_alpha': [0.0],
        }
    )

## Prediction

In [ ]:
def predict(name, isTrain=False):
    prefix = 'train' if isTrain else 'test'
    result = results[name]
    pred = result['model'].predict(result['X_'+prefix])
    result[prefix+'pred'] = pred

for name in results.keys():
    predict(name)
for name in results.keys():
    predict(name, isTrain=True)

In [ ]:
def predict_proba(name, isTrain=False):
    prefix = 'train' if isTrain else 'test'
    result = results[name]
    proba = result['model'].predict_proba(result['X_'+prefix])
    result[prefix+'pred_prob'] = proba

for name in results.keys():
    predict_proba(name)
for name in results.keys():
    predict_proba(name, isTrain=True)


## Evaluation

In [ ]:
def recall(cm, i):
    return cm[i][i]/sum(cm[i]) if sum(cm[i]) > 0 else np.inf

def precision(cm, i):
    cmt = np.copy(cm).transpose()
    return cm[i][i]/sum(cmt[i]) if sum(cmt[i]) > 0 else np.inf

def f_measure(cm, i):
    p = precision(cm, i)*100
    r = recall(cm, i)*100
    return 2 * (p * r) / (p + r) if p != np.inf and r != np.inf and p + r > 0 else np.nan

In [ ]:
from matplotlib.figure import Figure
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
def conf_matrix(y_test, y_pred, name, prefix):
    cm =  confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=['True', 'False'])
    disp.plot()
    disp.ax_.set_title(name + ' ' + prefix)

### Accuracy

In [ ]:
def score(name, isTrain=False):
    prefix = 'train' if isTrain else 'test'
    result = results[name]
    result[prefix+'score'] = result['model'].score(result['X_'+prefix], result['y_'+prefix])
    print(name.upper()+":\t", result[prefix+'score'])

print('\tAccuracy')
print("Test")
for name in results.keys():
    score(name)
print("\nTrain")
for name in results.keys():
    score(name, isTrain=True)

### Precision, Recall and F-measure

In [ ]:
def evaluate(name, isTrain=False):
    prefix = 'train' if isTrain else 'test'
    result = results[name]
    cm =  confusion_matrix(result['y_'+prefix], result[prefix+'pred'])
    print(prefix, name.upper()+":", '\tRecall:',round(recall(cm, 0), 2), '\t Precision:',round(precision(cm, 0), 2), '\tF_Measure:',round(f_measure(cm, 0), 2))

print('Test')
for name in results.keys():
    evaluate(name)
print('\nTrain')
for name in results.keys():
    evaluate(name, isTrain=True)


### Confusion Matrix

In [ ]:
prob = 0.50

prefixes = ('test', 'train')
for prefix in prefixes[:1]:
    for name in results.keys():
        result = results[name]
        prefix = 'test'
        conf_matrix(result['y_'+prefix], np.where(result[prefix+'pred_prob'][:,-1] > prob, 1, -1), name, prefix)

### AUC

In [ ]:
from sklearn.metrics import roc_curve, RocCurveDisplay, auc
import matplotlib.pyplot as plt

def roc_and_auc(name, isTrain=False):
    fig, ax = plt.subplots()
    for prefix in ('test', 'train'):
        result = results[name]
        fpr, tpr, thresholds = roc_curve(result['y_'+prefix], result[prefix+'pred_prob'][:,0], pos_label=result['model'].classes_[0])
        roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot(ax=ax,name=(name.upper()+' '+prefix))
        print(name.upper(), prefix + ":\t", auc(fpr, tpr))

for name in results.keys():
    roc_and_auc(name)